In [2]:
mssparkutils.notebook.run('4 - MSSpark Utilities/6 - Mount configuration')

In [3]:
## - Accessing the Files from Mountpoint 
## Syntax:
# synfs:/<jobid>/<mountpoint>/<path>
# To get JobID - mssparkutils.env.getJobId()


job_id = mssparkutils.env.getJobId()

mount_point = 'synfs:/' + job_id + '/lake'

In [4]:
## Reading the previous Schema data to a dataframe

df = spark.read.format('parquet')\
                .option('header','true')\
                .load(mount_point+'/SchemaManagement/*.parquet')



In [5]:
# Create normal multiply function in python 

def multiply2(x):
    return x*2

In [6]:
df.createOrReplaceTempView('SchemaView')

In [7]:
spark.sql("SELECT UnEmployed_Rate_Percentage, multiply2(UnEmployed_Rate_Percentage) AS Doubled FROM SchemaView").show()

In [8]:
# Registering the UDF to use in Spark SQL

spark.udf.register('multiply2_udf',multiply2)

In [9]:
spark.sql("SELECT UnEmployed_Rate_Percentage, multiply2_udf(UnEmployed_Rate_Percentage) AS Doubled FROM SchemaView").show()

In [10]:
# Applying the registered UDF on dataframe 

df.withColumn('multiplied',multiply2_udf(df.UnEmployed_Rate_Percentage)).show()

In [11]:
# Registering function to use in Dataframe 

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

multiply2_df_udf = udf(lambda x: multiply2(x), DoubleType())

In [13]:
display(df.withColumn('multiplied',multiply2_df_udf(df.UnEmployed_Rate_Percentage)))

# Method 2 - Using annotation

In [24]:
# Defining function using annotation 

@udf(DoubleType())
def add3(x):
    return x+3

In [25]:
display(df.withColumn('Added',add3(df.UnEmployed_Rate_Percentage)))